# Import Libraries

In [ ]:
#Download Data from CBOE
from selenium import webdriver
from bs4 import BeautifulSoup

#DataFrames manipulation
import pandas as pd
import numpy as np

#Tracking Loop Porgress
from tqdm import tqdm

#Datetime Utilities
from datetime import datetime
from datetime import timedelta
from pandas.tseries.offsets import CustomBusinessDay
from pandas.tseries.holiday import USFederalHolidayCalendar

#Data download from Yahoo Finance
from yahooquery import Ticker

#Function to Calculate the Term Structure's Slope
from sklearn.linear_model import LinearRegression

#Libraries for the Plotting
import holoviews as hv
from holoviews import opts, dim
from bokeh.models import HoverTool

#Librarie to save the plots to html object
import panel as pn
import param

hv.extension('bokeh')

# Get VIX Futures Term Structure from CBOE

In [ ]:
#*** Get all the Links related with VIX Futures from CBOE ***

#Set up the Selenium web driver
driver = webdriver.Chrome()  # You can use another browser's driver if you prefer

#URL of the webpage
cboe_link = "https://www.cboe.com/us/futures/market_statistics/historical_data/"

#Open the URL using the web driver
driver.get(cboe_link)

#Get the page source after JavaScript execution
page_source = driver.page_source

#Close the web driver
driver.quit()

#Parse the page source using BeautifulSoup
soup = BeautifulSoup(page_source, "html.parser")

#Find all <a> tags
links = soup.find_all("a")

#Filter and store the href attributes of the links starting with the specified prefix
prefix = "https://cdn.cboe.com/data/us/futures/market_statistics/historical_data/VX/"
filtered_links = [link.get("href") for link in links if link.get("href") and link.get("href").startswith(prefix)]

In [ ]:
#*** Download the csv files (data) from all the links obtained from CBOE website

#Create a New object for the Loop
data_frames = []

#Loop through the links and read each CSV into a DataFrame
for link in tqdm(filtered_links):
    df = pd.read_csv(link)
    data_frames.append(df)

#Concatenate all the DataFrames vertically
merged_df = pd.concat(data_frames, ignore_index=True)

In [ ]:
#Create New Object to Get the VIX Futures Curves
vix_futures = merged_df

#Limit the Dataframe for only the data required
vix_futures = vix_futures[['Trade Date', 'Futures', 'Settle']]

#Convert Trade Date to datetime format
vix_futures['Trade Date'] = pd.to_datetime(vix_futures['Trade Date'])


#Change the Values in column Futures to be in Datetime format
vix_futures['Futures'] = vix_futures['Futures'].str.extract(r'\((.*?)\)')
vix_futures['Futures'] =pd.to_datetime(vix_futures['Futures'], format='%b %Y')

#Create new row with the Months to Expiry
vix_futures['Trade Date Helper'] = vix_futures['Trade Date'].apply(lambda x: x.replace(day=1))
vix_futures['Months to Expiry'] = vix_futures.apply(lambda row: (row['Futures'].year - row['Trade Date Helper'].year) * 12 + row['Futures'].month - row['Trade Date Helper'].month, axis=1)

#Limit the dataframe for only the necessary columns
vix_futures = vix_futures[['Trade Date', 'Months to Expiry', 'Settle']]

In [ ]:
#Convert the table to Pivot
vix_futures = vix_futures.groupby(['Trade Date', 'Months to Expiry'])['Settle'].last().reset_index()
vix_futures = vix_futures.pivot(index='Trade Date', columns='Months to Expiry', values='Settle')

# Remove rows that have no values
rows_to_remove = vix_futures.apply(lambda row: all(pd.isna(v) or v == 0 for v in row[1:]), axis=1)
vix_futures = vix_futures[~rows_to_remove]

# Convert cells with value 0 to NaN
vix_futures = vix_futures.replace(0, np.nan)

# Create the Dataframe and Graph for the Most Recent VIX Futures Curve

In [ ]:
#Most Recent Observation
last_vix_futures = pd.DataFrame(vix_futures.iloc[-1]).dropna()

In [ ]:
#Instantiate the Dataframe for the Graph
graph_df = last_vix_futures.reset_index().rename(columns={last_vix_futures.columns[0]: 'VIX_Curve'})

#Create the Graph
last_vix_futures_graph = hv.Curve(graph_df).opts(opts.Curve(tools=['hover'])).opts(width=800, height=400, ylabel = 'VIX Level',color = 'red', xticks=10
                                                               ).relabel('VIX Term Structure Curve')

In [ ]:
#Save the Plots
p = pn.panel(last_vix_futures_graph)
p.save('Latest_VIX_Term_Structure_Curve_Graph.html', embed = True)

# Create the Dataframe and Graph for the VIX Futures Curves (Last, 1 Month Ago, 1 Year Ago)

In [ ]:
# Filter for the latest value
latest_date = vix_futures.index.max()
latest_value = vix_futures.loc[latest_date]

#Define the US business day frequency using the US Federal Holiday calendar
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())

# Filter for the value one month ago
one_month_ago = latest_date -  pd.DateOffset(months=1) - us_bd
one_month_ago_value = vix_futures.loc[one_month_ago]

# Filter for the value one year ago
one_year_ago = latest_date -  pd.DateOffset(months=12) - us_bd
one_year_ago_value = vix_futures.loc[one_year_ago]

# Create a new DataFrame with the filtered values
result_df = pd.concat([latest_value, one_month_ago_value, one_year_ago_value], axis=1)
result_df.columns = ['Latest', '1 Month Ago', '1 Year Ago']

In [ ]:
#Instantiate the Dataframe for the Graph
graph_df = result_df.dropna()
graph_df = graph_df.rename(columns = {'Months to Expiry': 'Months_to_Expiry', '1 Month Ago': '1_Month_Ago', 
                                      '1 Year Ago': '1_Year_Ago', })

#Generate all curves
def getCurves(n):
    for column in graph_df.columns:
        hover = HoverTool(tooltips=[("Months_to_Expiry", "@Months_to_Expiry"), ('VIX Level', f"@{column}")])  
        curve = hv.Curve(graph_df[column], label = column).opts(opts.Curve(tools=[hover]))
        yield curve
        
source_curves, target_curves  = [], []
for curve in getCurves(2):
    
    src = curve.relabel('').opts(width=800, height=100, yaxis=None) 
    tgt = curve.opts(width=800, ylabel = 'VIX Level', xlabel = 'Months to Expiry')
    source_curves.append(src)
    target_curves.append(tgt)
    
#Get the graph with the target curves
overlaid_plot_tgt = hv.Overlay(target_curves).relabel('VIX Term Structure Curve: Latest, 1 Month Ago, and 1 Year Ago').opts(
    height=400, legend_position='top')

# Layout the plot
vix_futures_curves_multiple_graph = overlaid_plot_tgt

In [ ]:
#Save the Plots
p = pn.panel(vix_futures_curves_multiple_graph)
p.save('VIX_Term_Structure_Curve_Multiple_Graph.html', embed = True)

# VIX Futures Curves Slope

In [ ]:
#Create New Object to Add the VIX Futures Slope
vix_futures_slope = vix_futures

slopes = []

#Calculate the slopes for each row using linear regression
for index, row in vix_futures.iterrows():
    valid_row = row.dropna()
    if len(valid_row) >= 2:
        x = valid_row.index
        y = valid_row.values
        x = np.array(x).reshape(-1, 1)
        y = np.array(y).reshape(-1, 1)

        model = LinearRegression()
        model.fit(x, y)
        slope = model.coef_[0][0]
        slopes.append(slope)
    else:
        slopes.append(np.nan)
        
#Add the Slope values to the DataFrame
vix_futures_slope['Slope'] = slopes

#Limit the DataFrame to the "Slope" columns and Round it to 2 decimal places
vix_futures_slope = pd.DataFrame(vix_futures_slope['Slope'])
vix_futures_slope = round(vix_futures_slope, 2)

In [ ]:
#Instantiate the Dataframe for the Graph
graph_df = vix_futures_slope.reset_index()

#Create the Graph
#hover = HoverTool(tooltips=[("Date", "@Date{%F}"), ('Effective_Number_Stocks', f"@Effective_Number_Stocks")], formatters={'@Date': 'datetime'})  
vix_futures_slope_graph = hv.Curve(graph_df).opts(opts.Curve(tools=['hover'])).opts(width=800, height=400, ylabel = 'VIX Curve Slope',color = 'blue', xticks=10
                                                               ).relabel('VIX Term Structure Slope')

#Add an Horizontal Line at 0
hline = hv.HLine(0).opts(color='black', line_width=1, line_dash='5 5')

#Combined Graph
vix_futures_slope_graph = vix_futures_slope_graph * hline

In [ ]:
#Save the Plots
p = pn.panel(vix_futures_slope_graph)
p.save('VIX_Term_Structure_Slope_Graph.html', embed = True)

# VIX Futures Curves Slope, Level and S&P 500 Drawdown

In [ ]:
#Download the Data from Yahoo Finance
tickers = Ticker('^VIX ^SPX')
df = tickers.history(start = '1970-01-01')
df = df.reset_index()
df = df[['symbol','date', 'close']]

#Pivot the Dataframe for S&P 500 and VIX to come as columns
df = df.pivot(index='date', columns='symbol', values='close')

# Reset Index and change columns names
df = df.reset_index()
df = df.rename(columns = {'date': 'Date', '^SPX': 'SP500', '^VIX': 'VIX'})

#Convert Date column to Datetime
df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(datetime.combine(x, datetime.min.time())))

In [ ]:
#Merge the VIX Futures Slope dataframe with the Dataframe with S&P 500 and VIX Values
vix_futures_slope = vix_futures_slope.reset_index().rename(columns = {'Trade Date': 'Date'})
full_df = vix_futures_slope.merge(df, on='Date', how='left')

#Calculate the Drawdown for the S&P500
full_df['Cummax'] = full_df['SP500'].cummax()
full_df['Drawdown'] = (full_df['SP500'] - full_df['Cummax']) / full_df['Cummax']
full_df['Drawdown'] = round(full_df['Drawdown'] * 100,2)

#Keep only Necessary Columns
full_df = full_df[['Date','VIX','Slope','Drawdown']]

In [ ]:
#Create the Graph for VIX
vix_graph = hv.Curve(full_df[['Date','VIX']]).opts(opts.Curve(tools=['hover'])).opts(width=800, height=200, ylabel = 'VIX Level',color = 'saddlebrown', xticks=10
                                                               ).relabel('VIX Index: Historical Levels')

vix_graph = vix_graph * hv.HLine(30).opts(color='black', line_width=1, line_dash='5 5')

#Create the Graph for VIX Slope
vix_slope_graph = hv.Curve(full_df[['Date','Slope']]).opts(opts.Curve(tools=['hover'])).opts(width=800, height=200, ylabel = 'VIX Slope',color = 'blue', xticks=10
                                                               ).relabel('VIX Term Structure Slope: Historical Levels')

vix_slope_graph = vix_slope_graph * hv.HLine(0).opts(color='black', line_width=1, line_dash='5 5')

#Create the Graph for S&P 500 Drawdown
sp500_dd_graph = hv.Curve(full_df[['Date','Drawdown']]).opts(opts.Curve(tools=['hover'])).opts(width=800, height=200, ylabel = 'S&P 500 Drawdown (%)',color = 'red', xticks=10
                                                               ).relabel('S&P 500® Index: Historical Drawdown')

sp500_dd_graph = sp500_dd_graph * hv.HLine(0).opts(color='black', line_width=1, line_dash='5 5')

#Overlay the 3 graphs
full_graph_final = (vix_graph + vix_slope_graph + sp500_dd_graph).cols(1)

In [ ]:
#Save the Plots
p = pn.panel(full_graph_final)
p.save('VIX_Term_Structure_Final_Graph.html', embed = True)